# RSA Broadcast attack
So, imagine that you use public exponent $e=3$ (it was quite popular a long time ago), but you only choose messages that wrap the modulus $N$, so there is no option of taking cubic root. Is it really safe? Well, there is a very simple scenario, that shows that it's not.
Imagine, that you have a client, that uses RSA to send messages to your server. But he also sends messages to several other servers. Say, 3. The same message gets encrypted three times: with your public key, with another guy public key and with one more key.
$$C_1=M^{3} mod N_1$$
$$C_2=M^{3} mod N_2$$
$$C_3=M^{3} mod N_3$$
It is next to impossible to break each $C_{i}$ on its own, but with three, Marvin can solve the puzzle.
## Chinese remainder theorem
If one knows the remainders of Euclidian division of an integer $n$ by several integers, then once can determine uniquely the remainder of the division of $n$ by the product of these integers, under the condition that the divisors are pairwise coprime.

So how can we use this theorem? First, we need to check, that all the divisors ($N_1, N_2, N_3$) are pairwise coprime. Well, if they are not coprime, than we can find the greatest commond divisor of non-coprime ones and factor them, which would allow us to decrypt the message. So let's assume that they are. This means, that we can find such $X$, that:
$$ X\lt N_1 N_2 N_3$$
$$ X = C_1 mod N_1 $$
$$ X = C_2 mod N_2 $$
$$ X = C_3 mod N_3 $$
and also such $X$ is unique. Let's look at $C=M^{3}$. Since $M\lt N_1$ and $M\lt N_2$ and $M\lt N_3$, then $C=M^3\lt N_1N_2N_3$. Also:
$$ C = C_1 mod N_1 $$
$$ C = C_2 mod N_2 $$
$$ C = C_3 mod N_3 $$

So by using Chinese Remainder Theorem we can find $C$, and all that's left is to take a cubic root. 
## How to find C

Let $N_i, i=1,k$ be divisors and $c_i, i=1,k$ their respective remainders. $N=N_1N_2...N_k$, а $M_i=N/N_i$
Then $C=(\sum_{i=0}^{k}C_iM_i(M_i^{-1}\space mod\space N_i))\space mod\space N$

Use the equation to find $C$ and get the flag from the three ciphertexts that you'll receive from the server. Good luck!

In [1]:
import socket
import re
from Crypto.Util.number import inverse,long_to_bytes,bytes_to_long
class VulnServerClient:
    def __init__(self,show=True):
        """Initialization, connecting to server"""
        self.s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.s.connect(('cryptotraining.zone',1339))
        if show:
            print (self.recv_until().decode())
    def recv_until(self,symb=b'\n>'):
        """Receive messages from server, by default till new prompt"""
        data=b''
        while True:
            
            data+=self.s.recv(1)
            if data[-len(symb):]==symb:
                break
        return data
    def get_public_keys(self,show=True):
        """Receive public keys from the server"""
        self.s.sendall('public\n'.encode())
        response=self.recv_until().decode()
        if show:
            print (response)
        e1=int(re.search('(?<=e1: )\d+',response).group(0))
        N1=int(re.search('(?<=N1: )\d+',response).group(0))
        e2=int(re.search('(?<=e2: )\d+',response).group(0))
        N2=int(re.search('(?<=N2: )\d+',response).group(0))
        e3=int(re.search('(?<=e3: )\d+',response).group(0))
        N3=int(re.search('(?<=N3: )\d+',response).group(0))
       
        return [(e1,N1),(e2,N2),(e3,N3)]
    
    def get_ciphertexts(self,show=True):
        """Receive ciphertexts from the server"""
        self.s.sendall('ciphertext\n'.encode())
        response=self.recv_until().decode()
        if show:
            print (response)
        c1=bytes_to_long(bytes.fromhex(re.search('(?<=ciphertext1: )[0-9a-f]+',response).group(0)))
        c2=bytes_to_long(bytes.fromhex(re.search('(?<=ciphertext2: )[0-9a-f]+',response).group(0)))
        c3=bytes_to_long(bytes.fromhex(re.search('(?<=ciphertext3: )[0-9a-f]+',response).group(0)))
        return (c1,c2,c3)
    
    def __del__(self):
        self.s.close()

In [2]:
vs=VulnServerClient()
pk_list=vs.get_public_keys()
(c1,c2,c3)=vs.get_ciphertexts()

Welcome to RSA broadcast task
Available commands:
help - print this help
public - show public keys
ciphertext - show ciphertexts 
quit - quit
>
e1: 3
N1: 20287982006618431876793244706487063574769448388426702838915722457901061849764724362603953179532539827554365329323483013276648891796378507103593000936891322846098718727133325953848182431476448546554772557085987908949429403596359635314342612889908898272272322173341141651567301687427226491229442385493785765699715986462483124423171652756203919879715705590771525305446788322512844427648822922682205388423707896633544989180321378196798302096862401020103125458117084856441433418990681274327810046291890889882496621395403208554291777227706389355365678885157011052465893070731243360783878242460400701935579682716345964783581
e2: 3
N2: 19461826656993775602233892694656909792660373232000384211810816270789889052589469816096210883392037679550505993511270364500797371312576866213808413522877454484246665882296260574836114845774508160454919015698629693496502